# PyKOALA Data Reduction Sequence (Standard stars)

This notebook contains the basic reduction steps that can be done with pyKOALA for the KOALA instrument.

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
from pykoala import __version__
import warnings
from astropy import units as u
# You may want to comment the following line
warnings.filterwarnings("ignore")
print("pyKOALA version: ", __version__)

In [ ]:
%load_ext autoreload
%autoreload 2

First, let's import a basic module to handle the RSS data:

In [ ]:
from pykoala.instruments.koala_ifu import koala_rss

The koala_rss is a *DataContainer* that will be used to store the RSS data and track all the changes applied to it.

Now let's load some data that we have partially reduced with 2dfdr. The target will be the spectrophotometric standard star HR7596:

In [ ]:
# List of RSS objects
std_star_rss = []
aaomega_arms = {'blue': 1, 'red': 2}
# Choose which arm of the spectrograph is going to be used
arm = 'red'
path_to_data = 'data'

for i in [28, 29, 30]:
    filename = f"{path_to_data}/27feb{aaomega_arms[arm]}00{i}red.fits"
    rss = koala_rss(filename)
    print(f"File {filename} corresponds to object {rss.info['name']}")
    std_star_rss.append(rss)

Now let us start applying some corrections to the data!

In this tutorial, we will consider the following corrections:
- Instrumental throughput
- Atmospheric extinction
- Telluric absorption
- Sky emission

Some of these corrections might not be relevant at a particular wavelength regime. For example, the blue arm of the AAOMega spectrograph 3600-5000 A is not affected by the telluric absorption.

# Corrections
## Instrumental throughput

In [ ]:
from pykoala.corrections.throughput import ThroughputCorrection

The throughput correction accounts for the differences in the efficiency of each fibre in the instrument. This effect also depends on the wavelength that we are using.

In pyKOALA (at least version <= 0.1.1) this can be computed from a set of input rss files that correspond to flat exposures as follows

In [ ]:
flat_rss = [koala_rss("data/combined_skyflat_red.fits")]
throughput_corr = ThroughputCorrection.from_rss(flat_rss, clear_nan=True,
                                                medfilt=10)
for i in range(len(std_star_rss)):
    std_star_rss[i] = throughput_corr.apply(std_star_rss[i])

We can assess the quality of our resulting throughput correction by using the built-in quality control plotting functions

In [ ]:
throughput_fig = throughput_corr.throughput.plot()

Each correction is recorded within the `history` attribute:

In [ ]:
std_star_rss[0].history.show()

## Atmospheric extinction

In [ ]:
from pykoala.corrections.atmospheric_corrections import AtmosphericExtCorrection

AtmosphericExtCorrection?

In [ ]:
atm_ext_corr = AtmosphericExtCorrection.from_text_file(AtmosphericExtCorrection.default_extinction)

In [ ]:
for i in range(len(std_star_rss)):
    std_star_rss[i] = atm_ext_corr.apply(std_star_rss[i])

## Telluric absorption

In [ ]:
from pykoala.corrections.sky import TelluricCorrection, combine_telluric_corrections
from pykoala.corrections.wavelength import TelluricWavelengthCorrection

There are two ways of estimating the Tellucir correction: using a default model or using an empirical approach from the data.

In this example, we will compute a telluric correction for each input RSS, and later we will combine all of them into a final one. Alternatively, the user might want to first combine all the RSS data, and then compute the effective telluric absorption correction.

In [ ]:
all_telluric_corrections = []
for i in range(len(std_star_rss)):
    telluric_correction, fig = TelluricCorrection.from_model(
        std_star_rss[i], plot=True, width=30)
    # Reopen the figure
    plt.figure(fig)
    telluric_correction.flag_data_container(
        std_star_rss[i], telluric_correction=telluric_correction.telluric_correction,
        wavelength=telluric_correction.wavelength)
    wavelength_correction, wave_offset_figs = TelluricWavelengthCorrection.from_rss(
    std_star_rss[i], median_smooth=5, oversampling=1, pol_fit_deg=2, 
    plot=True)

    # Recompute the telluric correction
    telluric_correction, fig = TelluricCorrection.from_model(
        std_star_rss[i], plot=True, width=30)
    # Apply the correction to the star
    std_star_rss[i] = telluric_correction.apply(std_star_rss[i])
    all_telluric_corrections.append(telluric_correction)




In [ ]:
from pykoala.corrections.wavelength import TelluricWavelengthCorrection


In [ ]:
# Create the final telluric correction and save the result

final_telluric_correction = combine_telluric_corrections(
    all_telluric_corrections, ref_wavelength=std_star_rss[0].wavelength)
final_telluric_correction.save(filename=f"products/telluric_correction_{arm}.dat")

plt.figure()
plt.subplot(111)
plt.plot(final_telluric_correction.wavelength, final_telluric_correction.telluric_correction, c='c')
plt.ylabel('Telluric correction')
plt.xlabel('Wavelength')
plt.ylim(0.95, 2.5)

## Sky emission

This is quite a difficult correction. In particular, KOALA does not count with auxiliary sky fibres that allow to estimate the sky brightness simultaneous to the acquisition of data. Therefore, the estimation of the sky contribution must be inferred from the science exposure or from offset sky frames taken between the observing sequence. 

At present, pyKOALA provides several ways to estimate a sky emission model... See the sky emission tutorial for a more detailed discussion.

In [ ]:
from pykoala.corrections import sky
sky.SkyFromObject?

In [ ]:
for i in range(len(std_star_rss)):
    skymodel = sky.SkyFromObject(std_star_rss[i], bckgr_estimator='mad', source_mask_nsigma=3, remove_cont=False)
    skycorrection = sky.SkySubsCorrection(skymodel)
    
    # Store the value of the RSS intensity before substraction
    intensity_no_sky = std_star_rss[i].intensity.copy()

    std_star_rss[i], _ = skycorrection.apply(std_star_rss[i])
    
    # Compare between the two versions of the data
    fig = plt.figure(figsize=(10, 4))
    ax = fig.add_subplot(121)
    mappable = ax.imshow(np.log10(std_star_rss[i].intensity.value),
                         origin='lower', aspect='auto',
                         interpolation='none', cmap='jet')
    plt.colorbar(mappable, ax=ax, label=f'{std_star_rss[i].intensity.unit}')
    ax = fig.add_subplot(122)
    mappable = ax.imshow(np.log10(intensity_no_sky / std_star_rss[i].intensity),
                         vmin=-0.7, vmax=.7, origin='lower', aspect='auto',
                         interpolation='none', cmap='jet')
    plt.colorbar(mappable, ax=ax, label=r'$\log_{10}(\frac{I_{nosky}}{I_{skycorr}})$')
    plt.subplots_adjust(wspace=0.3)


# Cubing

For the final part of this tutorial, now we will see how to combine a set of RSS data into a 3D datacube.

- (optional) The first step would consists of registering the data, i.e., account for the spatial offset between the different frames eigther produced by instrumental innacuracies or due to the application of dithering patterns.
- ADR correction. The data might be affected by atmospheric differential refraction, producing a wavelength-dependent shift of the image.
- Cube interpolation.

## Registration

The registration of RSS frames is part of the Astrometry correction module. To register a set of stardad star frames, we will use the function `AstrometryCorrection.register_centroids` method.

In [ ]:
from pykoala.corrections.astrometry import AstrometryCorrection

astrom_corr = AstrometryCorrection()

star_name = std_star_rss[0].info['name'].split()[0]
offsets, fig = astrom_corr.register_centroids(std_star_rss, object_name=star_name,
                                         qc_plot=True, centroider='gauss')
for offset in offsets:
    print("Offset (ra, dec) in arcsec: ", offset[0].to('arcsec'), offset[1].to('arcsec'))

for rss, offset in zip(std_star_rss, offsets):
    astrom_corr.apply(rss, offset=offset)

# Check that the corrections has been logged
rss.history.show()

In [ ]:
fig

## Atmospheric differential refraction

Accounting for this correction is way more easier with standard stars than with extended sources. The idea is to track the centroid of the star as function of wavelength to derive the offsets produced by the ADR.

In [ ]:
from pykoala.corrections.atmospheric_corrections import get_adr

In [ ]:
adr_corr_set = []

for rss in std_star_rss:
    adr_pol_ra, adr_pol_dec, fig = get_adr(rss, max_adr=0.5, pol_deg=2,
                                        plot=True)
    adr_corr_set.append([adr_pol_ra, adr_pol_dec])
    plt.show(plt.figure(fig))

## RSS cubing

In [ ]:
from pykoala.cubing import build_cube, build_wcs
from pykoala.plotting.utils import qc_cube

For interpolating RSS data into a 3D datacube we will make use of the function *build_cube*. This method requires as input:
- A list of RSS objects. 
- An `astropy.wcs.WCS` instance describing the dimensions of the cube, or a dictionary containing the basic information for initialising a `WCS`.
- The characteristic size of the kernel interpolation function expressed in arcseconds.
- A list containing the ADR correction for every RSS (it can contain None) in the form: [(ADR_ra_1, ADR_dec_1), (ADR_ra_2, ADR_dec_2), (None, None)]. Note that for the first two RSS we would be providing some corrections, while the latter would not be corrected.

To facilitate the creation of `WCS` objects, we provide a function `build_wcs`

In [ ]:
# Number of pixels along the three dimensions
datacube_shape = (std_star_rss[0].wavelength.size, 40, 60)
# Reference position along the three axes.
ref_position = (std_star_rss[0].wavelength[0],
                np.mean(std_star_rss[0].info['fib_ra']),
                np.mean(std_star_rss[0].info['fib_dec']))
# Spatial pixel scale size in deg
spatial_pixel_size = 1.0 << u.arcsec
# Spectral pixel scale in angstrom
spectral_pixel_size = std_star_rss[0].wavelength[1] - std_star_rss[0].wavelength[0]

wcs = build_wcs(datacube_shape=datacube_shape,
                reference_position=ref_position,
                spatial_pix_size=spatial_pixel_size,
                spectra_pix_size=spectral_pixel_size,
            )

In [ ]:
cube = build_cube(rss_set=std_star_rss,
                  wcs=wcs,
                  kernel_size_arcsec=1.0,
                  # Additional information that will be stored in the *info* dictionary
                  cube_info=dict(name=star_name))

To assess the performance of the cubing, we can run the QC method *qc_cube*

In [ ]:
qc_cube(cube)


# Flux calibration

The final step will consists of deriving the instrumental response function that converts the signal in counts into physical units accounting for the instrumental chromatic flux loss.

To build a `FluxCalibration` correction we can provide a path to a text file that already contains the response function:
```
flux_correction = FluxCalibration.from_text_file("path/to/file")
```
This file must contain two columns that correspond to the wavelength and response function, respectively.

Alternatively, we can estimate the response function by means of observations of standard stars.

- First, we can extract the spectra from the data by modelling the instrumental PSF using a Moffat or Gaussian model:
```
FluxCalibration.extract_stellar_flux(data_container)
```
- Then, we can estimate the response using:
```
FluxCalibration.get_response_curve(obs_wavelength, obs_spectra, reference_wavelength, reference_spectra)
```
See the methods documentation for details.

Finally, `FluxCalibration` includes a semi-automatic method (`FluxCalibration.auto`) for building the correction from a set of standard stars. In this example we will use a single star, but the final response function can combine the results from several objects. 

In [ ]:
from pykoala.corrections.flux_calibration import FluxCalibration

# Basic configuration for the extraction of the stellar spectra from the cube
# wave_range = None ---> use the whole range of wavelength
# wave_window = 50 ---> Number of resolution elements or "spectral pixels" to average
extract_args = dict(wave_range=None, wave_window=5, plot=True)
# Basic configuration for the estimation of the response function
# median_filter_n ---> Apply a median filter on the observed data prior fit.
# spline = True ---> Fit a spline
response_params = dict(pol_deg=None, spline=True, median_filter_n=20, spline_args={'s':10},
                           plot=True)

flux_cal_results, flux_corrections, master_flux_corr = FluxCalibration.auto(
    data=[cube],
    calib_stars=[cube.info['name']],
    fnames=None,
    extract_args=extract_args,
    response_params=response_params,
    combine=True)

In this example we have extrated the flux of a single star (HILT600) to build a `FluxCalibrationCorrection`.

In [ ]:
print(flux_cal_results['HILT600'].keys())
flux_cal_results['HILT600']['extraction']['figure']

In [ ]:
print(flux_cal_results['HILT600'].keys())
# Show the figure
flux_cal_results['HILT600']['response_fig']


In [ ]:
# Save the reponse function
master_flux_corr.save_response('products/response_HILT600_transfer_function.dat')